In [22]:
import tensorflow as tf
import effnetv2_model

MODEL = 'efficientnetv2-b3'
ckpt_path = r"F:\Lab\nfs\nsl\training-run-b3-adam\ckpt-269"
# Build model
tf.keras.backend.clear_session()
model = effnetv2_model.EffNetV2Model(model_name=MODEL)

model.compile('rmsprop', 'sparse_categorical_crossentropy', metrics=['accuracy'])
_ = model(tf.ones([1, 224, 224, 3]), training=False)
model.load_weights(ckpt_path)
cfg = model.cfg


In [23]:

# Run inference for a given image
import preprocessing

import pandas as pd
import os


def preprocess(path):
    img = tf.io.read_file(path)
    img = preprocessing.preprocess_image(
        img, cfg.eval.isize, is_training=False, augname=cfg.data.augname)
    return img


df = pd.read_csv(r"F:\Lab\datasets\fairface\fairface_label_val.csv")

df.file = df.file.apply(lambda x: os.path.join(
    r"F:\Lab\datasets\fairface\fairface-img-margin025-trainval", x))

races = df.race.unique()
genders = df.gender.unique()
history = []
for race in races:
    for gender in genders:
        df_filtered = df[df.gender == gender]
        df_filtered = df_filtered[df_filtered.race == race]
        label = int(gender == "Male")
        test_ds = tf.data.Dataset.from_tensor_slices(list(df_filtered.file))
        test_ds = test_ds.map(lambda x: preprocess(
            x), num_parallel_calls=tf.data.AUTOTUNE)
        labels_ds = tf.data.Dataset.from_tensor_slices(
            len(df_filtered) * [label])
        test_ds = tf.data.Dataset.zip((test_ds, labels_ds))
        test_ds = test_ds.batch(128)
        hist = model.evaluate(test_ds)
        history.append([race, gender, hist])


# logits = model(tf.expand_dims(image, 0), False)

# # Output classes and probability
# pred = tf.keras.layers.Softmax()(logits)
# idx = tf.argsort(logits[0])[::-1][:5].numpy()
# import ast
# classes = ast.literal_eval(open(labels_map, "r").read())
# for i, id in enumerate(idx):
#   print(f'top {i+1} ({pred[0][id]*100:.1f}%):  {classes[id]} ')
# from IPython import display
# display.display(display.Image(image_file))


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


7/7 [==============================] - 6s 312ms/step - loss: 0.7997 - accuracy: 0.9459
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


7/7 [==============================] - 2s 306ms/step - loss: 1.4892 - accuracy: 0.9017
INFO:tensorflow:Use default resize_bicubic.


...

6/6 [==============================] - 2s 360ms/step - loss: 1.8131 - accuracy: 0.8781
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


7/7 [==============================] - 2s 326ms/step - loss: 0.2174 - accuracy: 0.9828
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


4/4 [==============================] - 1s 264ms/step - loss: 1.4716 - accuracy: 0.8965


In [21]:
history

[['East Asian', 'Male', [0.8966054320335388, 0.9395109415054321]],
 ['East Asian', 'Female', [1.5804468393325806, 0.8926261067390442]],
 ['White', 'Male', [0.9198470115661621, 0.9385026693344116]],
 ['White', 'Female', [1.6984361410140991, 0.8878504633903503]],
 ['Latino_Hispanic', 'Male', [0.7877232432365417, 0.9432534575462341]],
 ['Latino_Hispanic', 'Female', [1.4217491149902344, 0.9048192501068115]],
 ['Southeast Asian', 'Male', [0.8697397112846375, 0.9442176818847656]],
 ['Southeast Asian', 'Female', [2.1105213165283203, 0.8514705896377563]],
 ['Black', 'Male', [0.823609471321106, 0.9424280524253845]],
 ['Black', 'Female', [4.123591423034668, 0.7199471592903137]],
 ['Indian', 'Male', [0.4614535868167877, 0.9681274890899658]],
 ['Indian', 'Female', [2.041430711746216, 0.8623853325843811]],
 ['Middle Eastern', 'Male', [0.5231021642684937, 0.9643296599388123]],
 ['Middle Eastern', 'Female', [1.024112343788147, 0.9217171669006348]]]